## Análisis de Afiliados | Reglas Operativas

Se realizo un Análisis de Información de appsflyer, el cual permitirá generar acciones operativas especificas con los afiliados para implementar reglas operativas que a su vez los dejen operar con seguridad a un menor costo operativo y les permita alinear eventos con los afiliados.

* Paso 1: Se identifico el porcentaje de instalaciones menores a 60 segundos.

* Paso 2: Se extrajeron los registros atipicos en eventos especificos.

#### Primea extracción de datos

Tomando en cuenta todos los registros.

In [4]:
import pandas as pd
import sqldf #Libreria de python que nos ayuda a introducir codigo SQL
import os
import plotly.express as px

pd.set_option('display.float_format', lambda x: '%.2f' % x) #Opcion para notación científica

#Definimos la ruta para poder trabajar con los diferentes Scripts de apoyo
os.chdir('C:/Users/crf005r/Downloads/Analisis Reglas Operativas')
os.listdir()

base = pd.read_csv("base_general.csv")
base.head()

,mes,dia,media_source,af_prt,clasificacion_1,clasificacion_2,suma_seg,min_seg,max_seg,registros
0,4,18,Facebook Ads,aztecamkt,+60 seg,+24 hrs,3039295638.00,90333.00,604873.00,9100
1,4,18,NaN,papaya,+60 seg,+24 hrs,952808735.00,90060.00,602849.00,3571
2,4,18,thingortwoeu_int,thingortwo,+60 seg,-5 hrs,119670873.00,64.00,21569.00,14815
3,4,18,squad_retencion,NaN,+60 seg,-5 hrs,860589.00,61.00,21480.00,618
4,4,18,squad_retencion,NaN,30-60 seg,-5 hrs,5682.00,31.00,59.00,128


##### Porcentaje de registros rangos de tiempo.

En promedio tenemos entre 3% (200,000) y 5% (300,000) de los registros en rangos de tiempo menores a 60 segundos.

In [29]:
tmp = base.groupby(["dia","clasificacion_1"]).agg({"registros":"sum"}).groupby(level=0).apply(lambda x:100 * x / float(x.sum())).reset_index()

tmp_1 = base.groupby(["dia","clasificacion_1"],as_index= False).agg({"registros":"sum"})

pd.concat([tmp,tmp_1],axis=1).iloc[:,[0,1,5,2]]

,dia,clasificacion_1,registros,registros
0,18,+60 seg,4954990,96.20
1,18,-30 seg,139829,2.71
2,18,30-60 seg,55950,1.09
3,19,+60 seg,6158662,96.28
4,19,-30 seg,166625,2.60
5,19,30-60 seg,71623,1.12
6,20,+60 seg,5645061,96.43
7,20,-30 seg,144482,2.47
8,20,30-60 seg,64327,1.10
9,21,+60 seg,5768193,96.40


#### Extrayendo registros que cumplen ciertos criterios.

    1. Menores a 60 segundos

    2. Campos is_primary_attribution = True

    3. Eventos: 
    - EG|cuentaCreada|pageView
    - EG|cuentaCreadaN4|pageView
    - CP:nuevoInmediato|creditoDepositado|pageView
    - CP:recompra|creditoDepositado|pageView
    - CP:renovacion|creditoDepositado|pageView
    - install

In [31]:
base = pd.read_csv("base_final.csv")

base.head()

,mes,dia,appsflyer_id,clasificacion_1,evento,is_primary_attribution,media_source,af_prt,conteo,Equipo,Nombre
0,4,29,1616388087750-8181818551662244932,30-60 seg,EG|cuentaCreadaN4|pageView,True,Facebook Ads,aztecamkt,1,Marketing,Marketing
1,4,19,1610061285094-7765719704339313535,30-60 seg,CP:recompra|creditoDepositado|pageView,True,aztecamkt,aztecamkt,1,Marketing,Marketing
2,4,23,1619239056756-985088,30-60 seg,EG|cuentaCreada|pageView,True,bt_group,vacio,1,afiliados,BT Group
3,4,30,1616449587672-3113723,-30 seg,CP:recompra|creditoDepositado|pageView,True,thingortwoeu_int,thingortwo,1,afiliados,Thing or Two
4,4,20,1618951290032-81380916293004222,30-60 seg,EG|cuentaCreada|pageView,True,LEADGENIOS,vacio,1,afiliados,Lead Genios


In [33]:
base.shape

(4350, 11)

#### Query para la extracción de información

Nota: El etiquetado de Equipo y Nombre se hace con el catalogo, esto para evitar que la query sea muy extensa.

```mysql
WITH base_original AS (
    SELECT 
        EXTRACT(MONTH FROM event_time_selected_timezone) AS mes,
        EXTRACT(DAY FROM event_time_selected_timezone) AS dia,
        appsflyer_id,
        media_source,
        af_prt,
        evento,
        is_primary_attribution,
        DATETIME_DIFF(TIMESTAMP(device_download_time_selected_timezone),     TIMESTAMP(attributed_touch_time_selected_timezone), SECOND) AS seg,
        DATETIME_DIFF(TIMESTAMP(device_download_time_selected_timezone), TIMESTAMP(attributed_touch_time_selected_timezone), HOUR) AS hrs
    FROM `bancoazteca-master-hub.appsflyer_data.appsflyer_events_data`  
    WHERE event_time_selected_timezone between '2021-04-30' and '2021-05-12'
        AND  evento IN("install","EG|cuentaCreada|pageView","EG|cuentaCreadaN4|pageView","CP:nuevoInmediato|creditoDepositado|pageView","CP:recompra|creditoDepositado|pageView","CP:renovacion|creditoDepositado|pageView")
        AND attributed_touch_time_selected_timezone NOT LIKE '%1900%' AND attributed_touch_time_selected_timezone NOT LIKE '%trifacta%' 
        AND device_download_time_selected_timezone NOT LIKE '%1900%' AND device_download_time_selected_timezone NOT LIKE '%trifacta%'
        AND is_primary_attribution = "True"),

etiquetado_rangos AS (
    SELECT mes, dia, appsflyer_id, media_source, af_prt, evento, is_primary_attribution, seg, hrs,
        CASE
            WHEN seg <= 30 THEN "-30 seg" 
            WHEN seg > 30 AND SEG <= 60 THEN "30-60 seg"
            WHEN seg > 60 THEN "+60 seg"
            END AS clasificacion_1
    FROM base_original)

SELECT 
mes,
dia,
appsflyer_id,
clasificacion_1,
evento,
is_primary_attribution,
media_source,
af_prt,
COUNT(*) as conteo 
FROM etiquetado_rangos 
WHERE clasificacion_1 IN ("-30 seg","30-60 seg")
GROUP BY 1,2,3,4,5,6,7,8
```